In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
# Reading the data
data = pd.read_csv('casestudy.csv', index_col=0)

In [3]:
# structure of the data
data.head()

,customer_email,net_revenue,year
0,nhknapwsbx@gmail.com,249.92,2015
1,joiuzbvcpn@gmail.com,87.61,2015
2,ukkjctepxt@gmail.com,168.38,2015
3,gykatilzrt@gmail.com,62.40,2015
4,mmsgsrtxah@gmail.com,43.08,2015


In [4]:
# Checking for missing values
data.isnull().values.any()

False

As observed, the data has no missing values

In [5]:
# Creating a dictionary of where key is the year and it's value is the subset of the data corresponsing to the year

years = sorted(list((set(data['year']))))
yearly_data = defaultdict(pd.DataFrame)
for year in years:
    yearly_data[year] = data[data['year']==year]
    
output_df = pd.DataFrame()

## I've made separate dataframes for outputs of every requirement and then combining later just for the sake of this assignment to show clarity and flow 
## In a real world scenario, I'd create one common dataframe for all the requirements to save MEMORY, time and redundancy

## Total revenue by year

In [7]:
revenue = []

#calculating revenue for every year
for year in years:
    total_rev = sum(yearly_data[year]['net_revenue'])
    revenue.append(total_rev)

yearly_rev = pd.DataFrame()
yearly_rev['year'] = years
yearly_rev['total_revenue'] = revenue
yearly_rev

,year,total_revenue
0,2015,29036749.19
1,2016,25730943.59
2,2017,31417495.03


## New Customer Revenue

In [9]:
new_cust_rev = ['nan']

# Calculating new customer revenue from the 2nd year since the 1st year i.e 2015 will have no prior information
for i in range(1,len(years)):
    curr_year_data = yearly_data[years[i]]
    
    # set of customers in current and previous years
    curr_cust = set(curr_year_data['customer_email'])
    prev_cust = set(yearly_data[years[i-1]]['customer_email'])
    
    # getting the list of new customers
    new_cust = list(curr_cust - prev_cust)
    
    # filtering out the rows having new customers
    new_rev_data = curr_year_data[curr_year_data['customer_email'].isin(new_cust)]
    new_cust_rev.append(sum(new_rev_data['net_revenue']))

new_rev = pd.DataFrame()
new_rev['year'] = years
new_rev['new_cust_revenue'] = new_cust_rev
new_rev

,year,new_cust_revenue
0,2015,nan
1,2016,18245491.01
2,2017,28776235.04


In [10]:
# Adding dataframe to the final output
output_df = pd.merge(yearly_rev, new_rev, on="year")

## Existing Customer Growth

In [13]:
existing_cust_growth = ['nan']

# Calculating new customer revenue from the 2nd year since the 1st year i.e 2015 will have no prior information
for i in range(1,len(years)):
    # getting the data of the current and previous years
    curr_year_data = yearly_data[years[i]]
    prev_year_data = yearly_data[years[i-1]]
    
    # getting the set of customers from current and previous years
    curr_cust = set(curr_year_data['customer_email'])
    prev_cust = set(prev_year_data['customer_email'])
    
    # finding existing customers by intersecting the above sets
    existing_cust = list(curr_cust.intersection(prev_cust))
    
    exist_cust_data = curr_year_data[curr_year_data['customer_email'].isin(existing_cust)]
    
    # finding existing customer growth by subtracting revenue of previous year from current year
    rev_exist_cust = sum(exist_cust_data['net_revenue'])
    rev_exit_cust_prev = sum(prev_year_data[prev_year_data['customer_email'].isin(existing_cust)]['net_revenue'])
    exist_cust_growth = rev_exist_cust - rev_exit_cust_prev
    
    existing_cust_growth.append(exist_cust_growth)

growth = pd.DataFrame()
growth['year'] = years
growth['existing_cust_growth'] = existing_cust_growth
growth

,year,existing_cust_growth
0,2015,nan
1,2016,20335.46
2,2017,20611.34


In [12]:
# Adding dataframe to the final output
output_df = pd.merge(output_df, growth, on="year")

## Revenue lost from attrition

In [14]:
#attrition = revenue from lost customers - revenue from gained customers
attrition = ['nan']

# Calculating new customer revenue from the 2nd year since the 1st year i.e 2015 will have no prior information
for i in range(1,len(years)):
    # getting the data of the current and previous years
    curr_year_data = yearly_data[years[i]]
    prev_year_data = yearly_data[years[i-1]]
    
    # getting the set of customers from current and previous years
    curr_cust = set(curr_year_data['customer_email'])
    prev_cust = set(prev_year_data['customer_email'])
    
    #list of customers lost and customers gained
    customers_lost = list(prev_cust - curr_cust)
    customers_gained = list(curr_cust - prev_cust)
    
    #gettting the data of the customers lost and customers gained
    cust_lost_data = prev_year_data[prev_year_data['customer_email'].isin(customers_lost)]
    cust_gained_data = curr_year_data[curr_year_data['customer_email'].isin(customers_gained)]
    
    #calculating attrition lost from the above stated formula
    curr_attrition = sum(cust_lost_data['net_revenue']) - sum(cust_gained_data['net_revenue'])
    attrition.append(curr_attrition)

attrition_revenue = pd.DataFrame()
attrition_revenue['year'] = years
attrition_revenue['attrition'] = attrition
attrition_revenue

,year,attrition
0,2015,nan
1,2016,3326141.059999
2,2017,-5665940.1


In [15]:
# Adding dataframe to the final output
output_df = pd.merge(output_df, attrition_revenue, on="year")

## Existing Customer Revenue Current Year

In [16]:
existing_cust_rev = ['nan']

# Calculating new customer revenue from the 2nd year since the 1st year i.e 2015 will have no prior information
for i in range(1,len(years)):
    # getting the data of the current and previous years
    curr_year_data = yearly_data[years[i]]
    prev_year_data = yearly_data[years[i-1]]
    
    # getting the set of customers from current and previous years
    curr_cust = set(curr_year_data['customer_email'])
    prev_cust = set(prev_year_data['customer_email'])
    
    # getting a list of the existing customers
    existing_cust = list(curr_cust.intersection(prev_cust))
    
    exist_cust_data = curr_year_data[curr_year_data['customer_email'].isin(existing_cust)]
    
    rev_exist_cust = sum(exist_cust_data['net_revenue'])

    existing_cust_rev.append(rev_exist_cust)

existing_cust_revenue = pd.DataFrame()
existing_cust_revenue['year'] = years
existing_cust_revenue['existing_cust_rev'] = existing_cust_rev
existing_cust_revenue

,year,existing_cust_rev
0,2015,nan
1,2016,7485452.58
2,2017,2641259.99


In [17]:
# Adding dataframe to the final output
output_df = pd.merge(output_df, existing_cust_revenue, on="year")

## Existing Customer Revenue Prior Year

In [18]:
exisiting_cust_rev_prev = ['nan']

# Calculating new customer revenue from the 2nd year since the 1st year i.e 2015 will have no prior information
for i in range(1,len(years)):
    # getting the data of the current and previous years
    curr_year_data = yearly_data[years[i]]
    prev_year_data = yearly_data[years[i-1]]
    
    # getting the set of customers from current and previous years
    curr_cust = set(curr_year_data['customer_email'])
    prev_cust = set(prev_year_data['customer_email'])
    
    #getting the list of existing customers
    existing_cust = list(curr_cust.intersection(prev_cust))
    
    exist_cust_data = curr_year_data[curr_year_data['customer_email'].isin(existing_cust)]
    
    rev_exit_cust_prev = sum(prev_year_data[prev_year_data['customer_email'].isin(existing_cust)]['net_revenue'])
    
    exisiting_cust_rev_prev.append(rev_exit_cust_prev)

exisiting_cust_rev_previous = pd.DataFrame()
exisiting_cust_rev_previous['year'] = years
exisiting_cust_rev_previous['exist_cust_prev_rev'] = exisiting_cust_rev_prev
exisiting_cust_rev_previous

,year,exist_cust_prev_rev
0,2015,nan
1,2016,7465117.12
2,2017,2620648.65


In [19]:
# Adding dataframe to the final output
output_df = pd.merge(output_df, exisiting_cust_rev_previous, on="year")

## Total Customers Current Year

In [20]:
total_customers_curr = []

#calculating total customers for every year
for year in years:
    #creating a set of unique customer emails
    customers = set(yearly_data[year]['customer_email'])
    total_customers_curr.append(len(customers))

total_customers_current = pd.DataFrame()
total_customers_current['year'] = years
total_customers_current['total_customers_curr'] = total_customers_curr
total_customers_current

,year,total_customers_curr
0,2015,231294
1,2016,204646
2,2017,249987


In [21]:
# Adding dataframe to the final output
output_df = pd.merge(output_df, total_customers_current, on="year")

## Total Customers Previous Year

In [22]:
total_customers_prev = ['nan']

# Calculating new customer revenue from the 2nd year since the 1st year i.e 2015 will have no prior information
for i in range(1, len(years)):
    # getting data of the previous year
    prev_year_data = yearly_data[years[i-1]]
    
    # creating a set of unique customer emails
    customers = set(prev_year_data['customer_email'])
    total_customers_prev.append(len(customers))

total_customers_previous = pd.DataFrame()
total_customers_previous['year'] = years
total_customers_previous['total_cust_prev_year'] = total_customers_prev
total_customers_previous

,year,total_cust_prev_year
0,2015,nan
1,2016,231294
2,2017,204646


In [23]:
# Adding dataframe to the final output
output_df = pd.merge(output_df, total_customers_previous, on="year")

## New Customers

In [24]:
new_cust = ['nan']

# Calculating new customer revenue from the 2nd year since the 1st year i.e 2015 will have no prior information
for i in range(1, len(years)):
    # getting the data of the current and previous years
    curr_year = yearly_data[years[i]]
    prev_year = yearly_data[years[i-1]]
    
    # getting new customers by subtracting previous year's customers from current year
    new_cust_list = set(curr_year['customer_email']) - set(prev_year['customer_email'])
    new_cust.append(len(new_cust_list))
    
new_customers = pd.DataFrame()
new_customers['year'] = years
new_customers['new_customers'] = new_cust
new_customers

,year,new_customers
0,2015,nan
1,2016,145062
2,2017,229028


In [25]:
# Adding dataframe to the final output
output_df = pd.merge(output_df, new_customers, on="year")

## Lost Customers

In [26]:
lost_cust = ['nan']

# Calculating new customer revenue from the 2nd year since the 1st year i.e 2015 will have no prior information
for i in range(1, len(years)):
    # getting the data of the current and previous years
    curr_year = yearly_data[years[i]]
    prev_year = yearly_data[years[i-1]]
    
    # getting lost customers by subtracting current year's customers from previous year
    lost_cust_list = set(prev_year['customer_email']) - set(curr_year['customer_email'])
    lost_cust.append(len(lost_cust_list))
    
lost_customers = pd.DataFrame()
lost_customers['year'] = years
lost_customers['lost_customers'] = lost_cust
lost_customers

,year,lost_customers
0,2015,nan
1,2016,171710
2,2017,183687


In [27]:
# Adding dataframe to the final output
output_df = pd.merge(output_df, lost_customers, on="year")

# Final Output

In [28]:
output_df

,year,total_revenue,new_cust_revenue,existing_cust_growth,attrition,existing_cust_rev,exist_cust_prev_rev,total_customers_curr,total_cust_prev_year,new_customers,lost_customers
0,2015,29036749.19,nan,nan,nan,nan,nan,231294,nan,nan,nan
1,2016,25730943.59,18245491.01,20335.46,3326141.059999,7485452.58,7465117.12,204646,231294,145062,171710
2,2017,31417495.03,28776235.04,20611.34,-5665940.1,2641259.99,2620648.65,249987,204646,229028,183687
